In [4]:
import seaborn as sns
import concurrent.futures
import argparse
import numpy as np
import subprocess
import os
from pathlib import Path
from typing import List, Optional
import json
import pandas as pd
from collections import defaultdict 
import yaml
import json
import gin
from typing import Dict, Any

import lib_analysis
import lib_biased_mnist


In [27]:
DF = lib_analysis.read_problem('biased_mnist')

In [5]:
models = [lib_biased_mnist.make_base_mlp_model() for _ in range(2)]

In [26]:
for v in models[0].trainable_variables:
    w = DF['weights'][0]["0"]
    print(w.keys())
    print(v.name)
    assert v.name in w

dict_keys(['dense/kernel:0', 'dense_1/kernel:0', 'dense_1/bias:0'])
dense/kernel:0
dict_keys(['dense/kernel:0', 'dense_1/kernel:0', 'dense_1/bias:0'])
dense/bias:0


AssertionError: 

In [13]:
DF['weights'][0]

{'0': {'dense/kernel:0': array([[-0.23744592],
         [-0.00233284]]),
  'dense_1/kernel:0': array([[-14.84159374]]),
  'dense_1/bias:0': array([-0.01760847])},
 '1': {'dense_2/kernel:0': array([[0.00252096],
         [0.24259086]]),
  'dense_3/kernel:0': array([[14.51949024]]),
  'dense_3/bias:0': array([0.00639545])}}